In [ ]:
# Cài đặt driver Neo4j và tree-sitter
!pip install neo4j tree-sitter -q

# Cài đặt các gói ngữ pháp ĐÃ BIÊN DỊCH SẴN
!pip install tree-sitter-python tree-sitter-java tree-sitter-javascript tree-sitter-typescript -q

!pip install -q -U sentence-transformers langchain langchain-openai

In [ ]:
import os
import glob
from neo4j import GraphDatabase
from kaggle_secrets import UserSecretsClient
from tree_sitter import Language, Parser,Query, QueryCursor
from sentence_transformers import SentenceTransformer # MỚI

# --- Phần 1: Tải Ngữ pháp Tree-sitter (Cách làm MỚI) ---
# Chúng ta import trực tiếp các hàm 'language' từ các gói đã cài
try:
    from tree_sitter_python import language as python_lang_loader
    from tree_sitter_java import language as java_lang_loader
    from tree_sitter_javascript import language as js_lang_loader
    from tree_sitter_typescript.tsx import language as tsx_lang_loader 
    
    print("Đang tải các đối tượng Ngôn ngữ (Language objects)...")
    
    PY_LANG = Language(python_lang_loader())
    JAVA_LANG = Language(java_lang_loader())
    JS_LANG = Language(js_lang_loader()) # Trình phân tích JS cũng xử lý JSX
    TS_LANG = Language(ts_lang_loader())
    TSX_LANG = Language(tsx_lang_loader())
    
    LANGUAGE_MAP = {
        '.py': (PY_LANG, 'python'),
        '.java': (JAVA_LANG, 'java'),
        '.js': (JS_LANG, 'javascript'), 
        '.jsx': (JS_LANG, 'javascript'), 
        '.ts': (TS_LANG, 'typescript'),
        '.tsx': (TSX_LANG, 'typescriptreact')
    }
    print("Đã tải 6 ngôn ngữ vào bộ nhớ.")

except ImportError as e:
    print(f"LỖI: Không thể import ngôn ngữ tree-sitter: {e}")
    print("Vui lòng đảm bảo bạn đã chạy cell 'Bước 0 (Đã sửa)' ở trên.")
    raise e
except Exception as e:
    print(f"Lỗi không xác định khi tải ngôn ngữ: {e}")
    raise e

In [ ]:
# --- Phần 2: Trình kết nối Neo4j (Database Connector) ---
# [ĐÃ CẬP NHẬT CHO SCHEMA MỚI]

class Neo4jGraphConstructor:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        print("Đã kết nối tới Neo4j AuraDB.")

    def close(self):
        self.driver.close()
        print("Đã ngắt kết nối khỏi Neo4j.")

    def run_cypher_query(self, query, params=None):
        # (Giữ nguyên)
        with self.driver.session() as session:
            try:
                result = session.run(query, params)
                return [record for record in result]
            except Exception as e:
                print(f"Lỗi truy vấn CyPER: {e}")
                return None

    def create_constraints_and_indexes(self):
        """
        Tạo Ràng buộc VÀ Chỉ mục cho schema mới.
        """
        print("Đang tạo các ràng buộc (constraints)...")
        # Ràng buộc cơ bản
        self.run_cypher_query("CREATE CONSTRAINT IF NOT EXISTS FOR (f:File) REQUIRE f.path IS UNIQUE")
        self.run_cypher_query("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Class) REQUIRE c.fqn IS UNIQUE")
        self.run_cypher_query("CREATE CONSTRAINT IF NOT EXISTS FOR (fn:Function) REQUIRE fn.fqn IS UNIQUE")
        self.run_cypher_query("CREATE CONSTRAINT IF NOT EXISTS FOR (m:Method) REQUIRE m.fqn IS UNIQUE")
        self.run_cypher_query("CREATE CONSTRAINT IF NOT EXISTS FOR (p:Placeholder) REQUIRE p.fqn IS UNIQUE")
        self.run_cypher_query("CREATE CONSTRAINT IF NOT EXISTS FOR (d:GeneratedDescription) REQUIRE d.fqn IS UNIQUE")

        # MỚI: Ràng buộc cho các node schema mới
        self.run_cypher_query("CREATE CONSTRAINT IF NOT EXISTS FOR (cd:ClassDefinition) REQUIRE cd.fqn IS UNIQUE")
        self.run_cypher_query("CREATE CONSTRAINT IF NOT EXISTS FOR (fd:FunctionDefinition) REQUIRE fd.fqn IS UNIQUE")
        self.run_cypher_query("CREATE CONSTRAINT IF NOT EXISTS FOR (md:MethodDefinition) REQUIRE md.fqn IS UNIQUE")
        self.run_cypher_query("CREATE CONSTRAINT IF NOT EXISTS FOR (a:Attribute) REQUIRE a.fqn IS UNIQUE")


        print("Đang tạo các chỉ mục (indexes)...")
        # FULL-TEXT INDEX (cho tên code elements)
        self.run_cypher_query("""
            CREATE FULLTEXT INDEX names IF NOT EXISTS
            FOR (n:Class|Function|Method|Attribute)
            ON EACH [n.name]
        """)

        # VECTOR INDEX (cho mô tả LLM)
        self.run_cypher_query("""
            CREATE VECTOR INDEX descriptions IF NOT EXISTS
            FOR (n:GeneratedDescription)
            ON (n.embedding)
            OPTIONS { indexConfig: {
                `vector.dimensions`: 384,
                `vector.similarity_function`: 'cosine'
            }}
        """)

    def ingest_data(self, nodes, relationships):
        # (Giữ nguyên)
        print(f"Đang ghi {len(nodes)} nodes và {len(relationships)} relationships...")
        query_nodes = """
        UNWIND $nodes AS node_data
        MERGE (n {fqn: node_data.fqn})
        ON CREATE SET n += node_data, n:Node
        ON MATCH SET n += node_data, n:Node
        WITH n, node_data
        CALL apoc.create.addLabels(n, [node_data.type]) YIELD node
        RETURN count(node)
        """
        self.run_cypher_query(query_nodes, params={'nodes': nodes})
        if relationships:
            query_rels = """
            UNWIND $relationships AS rel_data
            MATCH (source {fqn: rel_data.source_fqn})
            MATCH (target {fqn: rel_data.target_fqn})
            CALL apoc.create.relationship(source, rel_data.type, rel_data.properties, target) YIELD rel
            RETURN count(rel)
            """
            self.run_cypher_query(query_rels, params={'relationships': relationships})
        print("Ghi dữ liệu thành công.")

In [ ]:
# --- Phần 3: Trình phân tích Code (TreeSitterParser) ---
# [ĐÃ CẬP NHẬT ĐỂ KHỚP VỚI FIG. 3]

class TreeSitterParser:
    def __init__(self, file_path, code_content, lang_tuple):
        self.file_path = file_path
        self.code_content = code_content
        self.language_obj = lang_tuple[0]
        self.language_name = lang_tuple[1]
        self.parser = Parser()
        self.parser.language = self.language_obj

        self.nodes = []
        self.relationships = []

        # MỚI: Biến để theo dõi context (hàm/method/class đang phân tích)
        self.current_context_fqn = None

        # Cập nhật Queries: Thêm 'attribute'
        self.queries = {
            'python': {
                'class': "(class_definition name: (identifier) @class_name)",
                'function': "(function_definition name: (identifier) @function_name)",
                'method': "(class_definition body: (block (function_definition name: (identifier) @method_name) @method_node) @class_node)",
                'import_from': "(import_from_statement module_name: (dotted_name) @module)",
                'call': "(call function: (identifier) @call_name)",
                'docstring': """
                    (function_definition body: (block (expression_statement (string) @docstring)) @func)
                    (class_definition body: (block (expression_statement (string) @docstring)) @class)
                """,
                'attribute': "(class_definition body: (block (expression_statement (assignment left: (identifier) @attr_name)))) @class_node" # Basic Python class attribute
            },
            'java': {
                'class': "(class_declaration name: (identifier) @class_name)",
                'function': "(method_declaration name: (identifier) @function_name)", # Gộp
                'method': "(class_declaration body: (class_body (method_declaration name: (identifier) @method_name) @method_node) @class_node)",
                'import_from': "(import_declaration (dotted_name) @module)",
                'call': "(method_invocation name: (identifier) @call_name)",
                'docstring': "((method_declaration (block_comment) @docstring) @func)",
                'attribute': "(class_declaration body: (class_body (field_declaration declarator: (variable_declarator name: (identifier) @attr_name)))) @class_node" # Java field
            },
            'javascript': { # Dùng cho .js và .jsx
                'class': "(class_declaration name: (identifier) @class_name)",
                'function': "(function_declaration name: (identifier) @function_name)",
                'method': "(class_declaration body: (class_body (method_definition name: (property_identifier) @method_name) @method_node) @class_node)",
                'import_from': "(import_statement source: (string) @module)",
                'call': "(call_expression function: (identifier) @call_name)",
                'docstring': "((function_declaration (statement_block (expression_statement (string) @docstring))) @func)",
                'attribute': "(class_declaration body: (class_body (public_field_definition name: (property_identifier) @attr_name))) @class_node" # JS/TS class field
            },
             'typescriptreact': { # Dùng cho .tsx (TS dùng chung JS)
                 'class': "(class_declaration name: (type_identifier) @class_name)",
                 'function': "(function_declaration name: (identifier) @function_name)",
                 'method': "(class_declaration body: (class_body (method_definition name: (property_identifier) @method_name) @method_node) @class_node)",
                 'import_from': "(import_statement source: (string) @module)",
                 'call': "(call_expression function: (identifier) @call_name)",
                 'docstring': "((function_declaration (statement_block (expression_statement (string) @docstring))) @func)",
                 'attribute': "(class_declaration body: (class_body (public_field_definition name: (property_identifier) @attr_name))) @class_node"
            }
            # Lưu ý: TS thường dùng query của JS, TSreact cần query riêng
        }

    def get_node_text(self, node):
        return self.code_content[node.start_byte:node.end_byte]

    def parse(self):
        tree = self.parser.parse(bytes(self.code_content, "utf8"))
        root_node = tree.root_node

        # Set context ban đầu là file
        self.current_context_fqn = self.file_path

        self.nodes.append({
            'fqn': self.file_path, 'type': 'File',
            'name': os.path.basename(self.file_path), 'path': self.file_path
        })

        self._run_query(self.language_name, 'class', root_node)
        self._run_query(self.language_name, 'function', root_node)
        self._run_query(self.language_name, 'method', root_node)
        self._run_query(self.language_name, 'import_from', root_node)
        self._run_query(self.language_name, 'call', root_node)
        self._run_query(self.language_name, 'docstring', root_node)
        self._run_query(self.language_name, 'attribute', root_node) # MỚI

        return self.nodes, self.relationships

    # Cập nhật để theo dõi context FQN
    def _run_query(self, lang_name, query_type, root_node):
        query_str = self.queries.get(lang_name, {}).get(query_type)
        if not query_str: return

        try:
            query = Query(self.language_obj, query_str)  # Sử dụng constructor Query
            cursor = QueryCursor(query)
            captures_dict = cursor.captures(root_node)
            captures = [(node, capture_name) for capture_name, nodes in captures_dict.items() for node in nodes]
        except Exception as e:
            print(f"Lỗi tree-sitter query ({query_type}): {e}")
            return

        # --- Logic để xác định context FQN (hàm/method/class chứa) ---
        def get_context_fqn(target_node):
            current_node = target_node.parent
            while current_node:
                node_type = current_node.type
                name_node = current_node.child_by_field_name('name')
                if name_node:
                    name = self.get_node_text(name_node)
                    if node_type in ('function_definition', 'method_declaration', 'function_declaration', 'method_definition'):
                         # Giả sử hàm/method nằm trong file hoặc class đã biết FQN
                         # Tìm FQN của class cha (nếu có)
                         class_parent = current_node.parent
                         while class_parent and class_parent.type not in ('class_definition', 'class_declaration'):
                             class_parent = class_parent.parent
                         if class_parent:
                             class_name_node = class_parent.child_by_field_name('name')
                             if class_name_node:
                                 class_name = self.get_node_text(class_name_node)
                                 return f"{self.file_path}::{class_name}::{name}" # Method
                         return f"{self.file_path}::{name}" # Function
                    elif node_type in ('class_definition', 'class_declaration'):
                        return f"{self.file_path}::{name}" # Class
                current_node = current_node.parent
            return self.file_path # Mặc định là file

        # -----------------------------------------------------------

        temp_methods_in_class = {}

        for node, capture_name in captures:
            # Xác định context FQN cho capture hiện tại
            self.current_context_fqn = get_context_fqn(node)

            if query_type == 'class' and capture_name == 'class_name':
                self._handle_class(node, node.parent)

            elif query_type == 'function' and capture_name == 'function_name':
                 # Chỉ xử lý nếu nó không phải là method (đã được handle bởi query 'method')
                is_method = False
                temp_parent = node.parent.parent
                while temp_parent:
                    if temp_parent.type in ('class_definition', 'class_declaration'):
                        is_method = True
                        break
                    temp_parent = temp_parent.parent
                if not is_method:
                    self._handle_function(node, node.parent, None)


            elif query_type == 'method' and capture_name == 'method_name':
                class_node = next((n for n, c in captures if c == 'class_node' and n.start_byte <= node.start_byte and n.end_byte >= node.end_byte), None)
                if class_node:
                    class_name_node = class_node.child_by_field_name('name')
                    if class_name_node:
                        method_fqn = f"{self.file_path}::{self.get_node_text(class_name_node)}::{self.get_node_text(node)}"
                        if method_fqn not in temp_methods_in_class:
                            self._handle_function(node, node.parent, class_name_node)
                            temp_methods_in_class[method_fqn] = True

            elif query_type == 'import_from' and capture_name == 'module':
                 self._handle_import(node)

            elif query_type == 'call' and capture_name == 'call_name':
                self._handle_call(node) # Sử dụng self.current_context_fqn

            elif query_type == 'docstring':
                self._handle_docstring(node, capture_name, captures) # Gán vào Definition node

            elif query_type == 'attribute' and capture_name == 'attr_name': # MỚI
                 class_node = next((n for n, c in captures if c == 'class_node' and n.start_byte <= node.start_byte and n.end_byte >= node.end_byte), None)
                 if class_node:
                     class_name_node = class_node.child_by_field_name('name')
                     if class_name_node:
                         self._handle_attribute(node, class_name_node)


    # Cập nhật: Tạo thêm Definition Node
    def _handle_class(self, name_node, class_node):
        class_name = self.get_node_text(name_node)
        class_fqn = f"{self.file_path}::{class_name}"
        class_def_fqn = f"DEF::{class_fqn}"

        # 1. Tạo Node Class
        if not any(n['fqn'] == class_fqn for n in self.nodes):
            self.nodes.append({'fqn': class_fqn, 'type': 'Class', 'name': class_name})
            self.relationships.append({'source_fqn': self.file_path, 'target_fqn': class_fqn, 'type': 'DEFINES_CLASS', 'properties': {}})

        # 2. Tạo Node ClassDefinition
        if not any(n['fqn'] == class_def_fqn for n in self.nodes):
             self.nodes.append({'fqn': class_def_fqn, 'type': 'ClassDefinition', 'code': self.get_node_text(class_node)})
             # 3. Tạo Relationship HAS_DEFINITION
             self.relationships.append({'source_fqn': class_fqn, 'target_fqn': class_def_fqn, 'type': 'HAS_DEFINITION', 'properties': {}})


    # Cập nhật: Tạo thêm Definition Node
    def _handle_function(self, name_node, func_node, class_name_node):
        func_name = self.get_node_text(name_node)
        if class_name_node: # Method
            class_name = self.get_node_text(class_name_node)
            class_fqn = f"{self.file_path}::{class_name}"
            func_type = 'Method'; func_fqn = f"{class_fqn}::{func_name}"; rel_type = 'HAS_METHOD'; source_fqn = class_fqn
            def_type = 'MethodDefinition'; def_fqn = f"DEF::{func_fqn}"
        else: # Function
            func_type = 'Function'; func_fqn = f"{self.file_path}::{func_name}"; rel_type = 'DEFINES_FUNCTION'; source_fqn = self.file_path
            def_type = 'FunctionDefinition'; def_fqn = f"DEF::{func_fqn}"

        # 1. Tạo Node Function/Method
        if not any(n['fqn'] == func_fqn for n in self.nodes):
            self.nodes.append({'fqn': func_fqn, 'type': func_type, 'name': func_name})
            self.relationships.append({'source_fqn': source_fqn, 'target_fqn': func_fqn, 'type': rel_type, 'properties': {}})

        # 2. Tạo Node FunctionDefinition/MethodDefinition
        if not any(n['fqn'] == def_fqn for n in self.nodes):
            self.nodes.append({'fqn': def_fqn, 'type': def_type, 'code': self.get_node_text(func_node)})
            # 3. Tạo Relationship HAS_DEFINITION
            self.relationships.append({'source_fqn': func_fqn, 'target_fqn': def_fqn, 'type': 'HAS_DEFINITION', 'properties': {}})


    def _handle_import(self, module_node):
        # (Giữ nguyên, vẫn dùng Placeholder)
        module_name = self.get_node_text(module_node).strip('\'\"')
        target_fqn = f"PLACEHOLDER_MODULE::{module_name}"
        if not any(n['fqn'] == target_fqn for n in self.nodes):
             self.nodes.append({'fqn': target_fqn, 'type': 'Placeholder', 'name': module_name})
        self.relationships.append({'source_fqn': self.file_path, 'target_fqn': target_fqn, 'type': 'IMPORTS', 'properties': {'module': module_name}})

    # Cập nhật: Nguồn là context hiện tại
    def _handle_call(self, call_node):
        call_name = self.get_node_text(call_node)
        target_fqn = f"PLACEHOLDER_CALL::{call_name}"
        if not any(n['fqn'] == target_fqn for n in self.nodes):
            self.nodes.append({'fqn': target_fqn, 'type': 'Placeholder', 'name': call_name})

        source_fqn = self.current_context_fqn if self.current_context_fqn else self.file_path
        self.relationships.append({'source_fqn': source_fqn, 'target_fqn': target_fqn, 'type': 'CALLS', 'properties': {'name': call_name}})

    # Cập nhật: Gán vào Definition node
    def _handle_docstring(self, docstring_node, capture_name, captures):
        docstring_text = self.get_node_text(docstring_node).strip('\'\"')
        parent_node_capture_name = 'func' if 'func' in [c[1] for c in captures] else 'class'
        parent_node = next((n for n, c in captures if c == parent_node_capture_name and n.start_byte <= docstring_node.start_byte and n.end_byte >= docstring_node.end_byte), None)

        if parent_node:
            name_node = parent_node.child_by_field_name('name')
            if name_node:
                parent_name = self.get_node_text(name_node)
                # Xác định FQN của node cha (Class/Function/Method)
                parent_fqn = None
                is_method = False
                temp_p = parent_node.parent
                while temp_p:
                     if temp_p.type in ('class_definition', 'class_declaration'):
                         class_name_node = temp_p.child_by_field_name('name')
                         if class_name_node:
                             parent_fqn = f"{self.file_path}::{self.get_node_text(class_name_node)}::{parent_name}" # Method
                             is_method = True
                         break
                     temp_p = temp_p.parent
                if not is_method:
                     parent_fqn = f"{self.file_path}::{parent_name}" # Class or Function

                if parent_fqn:
                    # Tìm FQN của Definition node tương ứng
                    def_fqn = f"DEF::{parent_fqn}"
                    # Tìm Definition node trong danh sách self.nodes và gán docstring
                    for node_data in self.nodes:
                        if node_data.get('fqn') == def_fqn:
                            node_data['docstring'] = docstring_text
                            break

    # HÀM MỚI
    def _handle_attribute(self, name_node, class_name_node):
        attr_name = self.get_node_text(name_node)
        class_name = self.get_node_text(class_name_node)
        class_fqn = f"{self.file_path}::{class_name}"
        attr_fqn = f"{class_fqn}::{attr_name}"

        # 1. Tạo Node Attribute
        if not any(n['fqn'] == attr_fqn for n in self.nodes):
            self.nodes.append({'fqn': attr_fqn, 'type': 'Attribute', 'name': attr_name})
            # 2. Tạo Relationship HAS_ATTRIBUTE
            self.relationships.append({'source_fqn': class_fqn, 'target_fqn': attr_fqn, 'type': 'HAS_ATTRIBUTE', 'properties': {}})

In [ ]:
# --- Phần 3.5: Trình làm giàu Code (Code Enricher) ---

class CodeEnricher:
    def __init__(self, llm, encoder):
        self.llm = llm
        self.encoder = encoder
        print("Trình làm giàu (Enricher) đã sẵn sàng.")

    def get_llm_description(self, code, docstring, node_type):
        # (Giữ nguyên)
        prompt = f"""
        Dựa trên {node_type} sau đây, hãy tạo một mô tả ngắn gọn (1-2 câu)
        giải thích mục đích chức năng của nó là gì.

        DOCSTRING (nếu có):
        {docstring}

        CODE:
        {code}

        MÔ TẢ NGẮN GỌN:
        """
        try:
            response = self.llm.invoke(prompt)
            return response.content.strip()
        except Exception as e:
            print(f"Lỗi LLM: {e}")
            return None

    def get_embedding(self, text):
        # (Giữ nguyên)
        return self.encoder.encode(text, convert_to_numpy=True).tolist()

    def enrich_nodes(self, nodes_list, relationships_list):
        """
        Duyệt qua các Definition node, tạo mô tả và embedding.
        """
        print("Bắt đầu làm giàu (enriching) các node Definition... (việc này có thể mất vài phút)")

        # Tìm các Definition node đã được tạo bởi Parser
        definition_nodes = [n for n in nodes_list if n['type'].endswith('Definition')]
        processed_count = 0

        for def_node in definition_nodes:
            # Lấy thông tin từ Definition node
            def_fqn = def_node['fqn']
            code = def_node.get('code', '')
            docstring = def_node.get('docstring', '')

            # Xác định loại node gốc (Function, Class, Method) từ FQN
            original_fqn = def_fqn.replace("DEF::", "")
            original_node_type = "Unknown"
            # Tìm node gốc để xác định loại (cách này hơi chậm nhưng đảm bảo)
            for n in nodes_list:
                 if n['fqn'] == original_fqn:
                      original_node_type = n['type']
                      break

            # 1. Gọi LLM
            description = self.get_llm_description(code, docstring, original_node_type)
            if not description:
                continue

            # 2. Tạo Embedding
            embedding = self.get_embedding(description)

            # 3. Tạo Node GeneratedDescription (FQN cần duy nhất)
            desc_fqn = f"DESC::{def_fqn}" # FQN dựa trên Definition node
            desc_node = {
                'fqn': desc_fqn,
                'type': 'GeneratedDescription',
                'description': description,
                'embedding': embedding
            }
            # Chỉ thêm nếu chưa tồn tại
            if not any(n['fqn'] == desc_fqn for n in nodes_list):
                 nodes_list.append(desc_node)

            # 4. Tạo Relationship HAS_DESCRIPTION (Từ Definition -> Description)
            desc_rel = {
                'source_fqn': def_fqn, # Nguồn là Definition node
                'target_fqn': desc_fqn,
                'type': 'HAS_DESCRIPTION',
                'properties': {}
            }
             # Chỉ thêm nếu chưa tồn tại
            if not any(r['source_fqn'] == def_fqn and r['type'] == 'HAS_DESCRIPTION' for r in relationships_list):
                 relationships_list.append(desc_rel)
                 processed_count += 1

        print(f"Làm giàu thành công! Đã thêm {processed_count} node mô tả.")

In [ ]:
# --- Phần 4: Chạy Pipeline Giai đoạn 1 (Đã sửa) ---
from langchain_openai import ChatOpenAI
try:
    # 1. Chỉ định đường dẫn
    REPO_ROOT_DIR = '/kaggle/input/chatbot/chatbotai_v1-main/' 
    
    if not os.path.exists(REPO_ROOT_DIR):
        print(f"LỖI: Đường dẫn '{REPO_ROOT_DIR}' không tồn tại.")
        raise FileNotFoundError(f"{REPO_ROOT_DIR} not found.")

    # 2. Lấy secrets từ Kaggle
    secrets = UserSecretsClient()
    NEO4J_URI = secrets.get_secret("NEO4J_URI")
    NEO4J_USER = secrets.get_secret("NEO4J_USER")
    NEO4J_PASSWORD = secrets.get_secret("NEO4J_PASSWORD")
    
    OPENAI_API_KEY = secrets.get_secret("OPENAI_API_KEY")
    graph_constructor = Neo4jGraphConstructor(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    
    llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o")
    
    encoder = SentenceTransformer('all-MiniLM-L6-v2') 
    enricher = CodeEnricher(llm, encoder)

    # 4. Xóa sạch DB (để chạy demo) và TẠO CONSTRAINTS & INDEXES
    print("Đang dọn dẹp cơ sở dữ liệu (nếu có)...")
    graph_constructor.run_cypher_query("MATCH (n) DETACH DELETE n")
    graph_constructor.create_constraints_and_indexes() 
    
    all_nodes = []
    all_relationships = []

    # 5. "Identify Files to Parse" (Đa ngôn ngữ)
    print(f"Bắt đầu quét {REPO_ROOT_DIR} cho các file đa ngôn ngữ...")
    search_patterns = [os.path.join(REPO_ROOT_DIR, '**', f'*{ext}') for ext in LANGUAGE_MAP.keys()]
    repo_files = []
    for pattern in search_patterns:
        repo_files.extend(glob.glob(pattern, recursive=True))

    print(f"Tìm thấy {len(repo_files)} file. Bắt đầu phân tích (Parse)...")

    for file_path in repo_files:
        file_ext = os.path.splitext(file_path)[1]
        lang_tuple = LANGUAGE_MAP.get(file_ext)
        if not lang_tuple: continue
        
        try:
            with open(file_path, "r", encoding='utf-8') as f: code_content = f.read()
        except Exception as e:
            print(f"Lỗi khi đọc file {file_path}: {e}. Bỏ qua.")
            continue
        
        # 6. "Parse", "Identify Components", "Align with Schema"
        print(f"Đang phân tích ({lang_tuple[1]}): {file_path}...")
        try:
            parser = TreeSitterParser(file_path, code_content, lang_tuple)
            nodes, relationships = parser.parse()
            all_nodes.extend(nodes)
            all_relationships.extend(relationships)
        except Exception as e:
            print(f"Lỗi khi phân tích cú pháp file {file_path}: {e}. Bỏ qua.")
            continue

    print(f"Phân tích hoàn tất. Tổng cộng {len(all_nodes)} nodes và {len(all_relationships)} relationships.")

    # 7. MỚI: "Enrichment" (LLM Descriptions & Embeddings)
    enricher.enrich_nodes(all_nodes, all_relationships)

    # 8. "Construct Knowledge Graph" (Ghi vào DB)
    if all_nodes:
        graph_constructor.ingest_data(all_nodes, all_relationships)
    else:
        print("Không có node nào để ghi vào Neo4j.")

    # Đóng kết nối
    graph_constructor.close()
    
    print("\n--- HOÀN TẤT GIAI ĐOẠN 1 (HOÀN CHỈNH) ---")
    print("Đồ thị tri thức đã được XÂY DỰNG, LÀM GIÀU, và LẬP CHỈ MỤC.")
    print("Sẵn sàng cho Giai đoạn 2 (Graph Based Code Retrieval) với LangGraph.")

except Exception as e:
    print(f"\nĐÃ XẢY RA LỖI:")
    print(e)
    if 'secrets' not in locals():
        print("Vui lòng kiểm tra lại 'Secrets' (NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD, GOOGLE_API_KEY).")